In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.client import device_lib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow

In [19]:
train = open(r'HerMajestySpeechesDataset\train.txt', 'r')
val = open(r'HerMajestySpeechesDataset\dev.txt', 'r')
test = open(r'HerMajestySpeechesDataset\test.txt', 'r')